# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
import findspark
findspark.init('/home/fernando/spark-2.4.6-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [3]:
data = spark.read.csv('Recommender_Systems/movielens_ratings.csv',inferSchema=True,header=True)

In [4]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [6]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [7]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [8]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [9]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|   1.0|    26|   3.2156796|
|     31|   1.0|    27|   1.1197516|
|     85|   3.0|    21|    4.159607|
|     65|   2.0|     3|    2.335976|
|     65|   2.0|     5|   3.9475577|
|     65|   1.0|     4|  0.92558944|
|     53|   2.0|    19|   1.7378783|
|     53|   1.0|     9|   2.1849096|
|     53|   1.0|    25|   -1.795626|
|     78|   1.0|    22|   0.5618689|
|     78|   1.0|    20| -0.24391827|
|     78|   1.0|    24|   1.1150742|
|     34|   1.0|    16|   1.5903728|
|     34|   1.0|     0| -0.16496116|
|     81|   2.0|     9|   2.8557782|
|     28|   1.0|     6|    4.141017|
|     28|   1.0|    10| -0.65786326|
|     76|   1.0|    20|-0.118831046|
|     76|   5.0|    14|   1.7609266|
|     26|   1.0|     2|  0.18747246|
+-------+------+------+------------+
only showing top 20 rows



In [10]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.104197820864312


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [11]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [12]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     16|    11|
|     20|    11|
|     25|    11|
|     32|    11|
|     35|    11|
|     36|    11|
|     47|    11|
|     50|    11|
|     79|    11|
|     82|    11|
|     86|    11|
|     89|    11|
+-------+------+



In [13]:
reccomendations = model.transform(single_user)

In [14]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     89|    11|  3.4184656|
|     50|    11|  1.7789286|
|      9|    11|  1.4307777|
|     32|    11|  1.0167427|
|     36|    11| 0.62360114|
|     20|    11|  0.4545604|
|     79|    11| 0.37307703|
|     82|    11|  0.3629592|
|     35|    11|  0.2753843|
|     86|    11| 0.07199755|
|     16|    11|-0.92474043|
|     25|    11|  -2.616991|
|     47|    11|  -3.607367|
+-------+------+-----------+



# Great Job!